# Reinforcement Learning Solution to the Towers of Hanoi Puzzle

In [4]:
import neuralnetworks as nnQ
import random
import numpy as np

In [103]:
from statistics import mean 

In [303]:
from copy import copy
from copy import deepcopy

## Requirements

First, how should we represent the state of this puzzle?  We need to keep track of which disks are on which pegs. Name the disks 1, 2, and 3, with 1 being the smallest disk and 3 being the largest. The set of disks on a peg can be represented as a list of integers.  Then the state can be a list of three lists.

For example, the starting state with all disks being on the left peg would be `[[1, 2, 3], [], []]`.  After moving disk 1 to peg 2, we have `[[2, 3], [1], []]`.

To represent that move we just made, we can use a list of two peg numbers, like `[1, 2]`, representing a move of the top disk on peg 1 to peg 2.

Now on to some functions. Define at least the following functions. Examples showing required output appear below.

   - `printState(state)`: prints the state in the form shown below
   - `validMoves(state)`: returns list of moves that are valid from `state`
   - `makeMove(state, move)`: returns new (copy of) state after move has been applied.
   - `trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF)`: train the Q function for number of repetitions, decaying epsilon at start of each repetition. Returns Q and list or array of number of steps to reach goal for each repetition.
   - `testQ(Q, maxSteps, validMovesF, makeMoveF)`: without updating Q, use Q to find greedy action each step until goal is found. Return path of states.

A function that you might choose to implement is

   - `stateMoveTuple(state, move)`: returns tuple of state and move.  
    
This is useful for converting state and move to a key to be used for the Q dictionary.

Show the code and results for testing each function.  Then experiment with various values of `nRepetitions`, `learningRate`, and `epsilonDecayFactor` to find values that work reasonably well, meaning that eventually the minimum solution path of seven steps is found consistently.

Make a plot of the number of steps in the solution path versus number of repetitions. The plot should clearly show the number of steps in the solution path eventually reaching the minimum of seven steps, though the decrease will not be monotonic.  Also plot a horizontal, dashed line at 7 to show the optimal path length.

Add markdown cells in which you describe the Q learning algorithm and your implementation of Q learning as applied to the Towers of Hanoi problem.  Use at least 15 sentences, in one or more markdown cells.

# Functions
The following functions are used to implement the Q learning algorithm applied to the Towers of Hanoi problem.
Each function should have a brief description on how what it does and how it relates to the Q learning algorithm.

The following function is a helper function, which turns state and move into tuples, this is necessary for iterating over them in the Dictionary Q, it will return the "tuplefide" state and move pair.

In [309]:
def stateMoveTuple(state, move):
    tempTupState = []
    for x in state:
        tempTupState.append(tuple(x))        
    return (tuple(tempTupState),tuple(move))

In [13]:
def validMoves(state):
    validMoves = []
    for x in range(1,4):
        for y in range(1,4):
            if len(state[x-1]) > 0:
                topx = state[x-1]
                if len(state[y-1]) !=0:
                    topy = state[y-1]
                if len(state[y-1]) == 0:
                    validMoves.append([x,y])
                elif((topx != topy and topy > topx)):
                    validMoves.append([x,y])
                elif((topx != topy and topy > topx)):
                    validMoves.append([x,y])
                elif((topx != topy and topy > topx)):
                    validMoves.append([x,y])

    return validMoves                

In [380]:
def printState(state):
    temp = 0
    col1Len = 0
    col2Len = 0
    col3Len = 0
    big = len(max(state))
    while temp < len(max(state)):
        holder1,holder2,holder3 = " "," "," "
        
        if col1Len< len(state[0]) and col1Len !=big and col1Len == temp:
            holder1 = state[0][temp]
            col1Len = col1Len + 1
        if col2Len < len(state[1]) and col2Len == temp:
            holder2 = state[1][temp]
            col2Len = col2Len + 1
        if col3Len < len(state[2])and col3Len == temp:
            holder3 = state[2][temp]
            col3Len = col3Len + 1
            
        print(holder1,holder2,holder3)
        temp = temp+1
    print("------")

In [302]:
def makeMove(state, move):
    state2 = deepcopy(state)
    temp = state2[move[0]-1][0]
    state2[move[0]-1].pop(0)
    state2[move[1]-1].insert(0,temp)
    return state2

The trainQ function 

In [345]:
def trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF):
    Q = {}
    
    state = [[1,2,3],[],[]]
    epsilon = 1
    step_count = []
    
    for x in range(nRepetitions):
        
        epsilon = epsilonDecayFactor * epsilon
        step = 0
        goal_state = [[],[],[1,2,3]]
        done = False
        
        while done != True:
            
            step = step + 1
            next_move = epsilonFindMoves(Q, state, epsilon, validMovesF)
            #print(state,next_move)
            next_state = makeMove(state,next_move) 
            
            if stateMoveTuple(state,next_move) not in Q:
                Q[stateMoveTuple(state,next_move)] = 0
                
            if next_state == goal_state:
                Q[stateMoveTuple(state,next_move)] = 1
                done = True
                
            if step > 1:
                Q[stateMoveTuple(stateOld,moveOld)] += learningRate *(1+Q[stateMoveTuple(state,next_move)]-
                                                        Q[stateMoveTuple(stateOld,moveOld)])
            stateOld, moveOld = state, next_move
            state = next_state
        state = [[1,2,3],[],[]]
        step_count.append(step)
    return Q ,step_count

In [355]:
def testQ(Q, maxSteps, validMovesF, makeMovesF):
    
    state = [[1,2,3],[],[]]
    epsilon = 0
    path = []
    #path.append(state)
    
    step = 0
    goal_state = [[],[],[1,2,3]]
    done = False

    while done != True:
        step = step+1
        path.append(state)
        next_move = epsilonFindMoves(Q, state, epsilon, validMovesF)

        next_state = makeMove(state,next_move) 
        

        if next_state == goal_state:
            done = True
        if step > maxSteps:
            done = True
        state = next_state
    return path

The epsilonFindMoves(...) function takes in the 

In [301]:
def epsilonFindMoves(Q, state, epsilonRate, validMovesF):
    validMoveList = validMoves(state)
    if np.random.uniform()<epsilonRate:
        return validMoveList[np.random.choice(len(validMoveList))] 
    else:
        Qs = np.array([Q.get(stateMoveTuple(state, m), 0) for m in validMoveList])
        return validMoveList[np.argmin(Qs)]

In [369]:
for x in path:
    printState(x)

1    
2    
3    
------
2   1
3    
------
3 2 1
------
3 1  
------
  1 3
------
1 2 3
------
1   2
    3
------


In [370]:
Q, stepsToGoal = trainQ(1000, 0.5, 0.7, validMoves, makeMove)

In [371]:
mean(stepsToGoal)

7.599

In [372]:
mean(stepsToGoal)

7.599

# Testing

In [378]:
state = [[1,2,3],[],[]]
move = [1,2]

In [377]:
#testing stateMoveTuple(state,move)
print(stateMoveTuple(state,move))

(((1, 2, 3), (), ()), (1, 2))


In [376]:
#testing validMoves(state)
print(validMoves(state))

[[1, 2], [1, 3]]


In [383]:
#testing printState(state)
printState(state)

1    
2    
3    
------


In [384]:
#testing makeMove(state,move)
print(makeMove(state,move))

[[2, 3], [1], []]


In [399]:
#testing trainQ(...)
Q, stepsToGoal = trainQ(50, 0.5, 0.7, validMoves, makeMove)
print('State Actions in Q:' ,len(Q))
print("Progression of steps to goal:")
print(np.array(stepsToGoal))
print('Mean of steps:',mean(steps))

State Actions in Q: 76
Progression of steps to goal:
[ 95  37 237  19  23  45  26  23   7  31  14  44   8  26  10   9   8  14
   7  32  11   7  16   7   7   7   7   7   9   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7]
Mean of steps: 7.578


In [410]:
#testing testQ(...)
Q, stepsToGoal = trainQ(100, 0.5, 0.7, validMoves, makeMove)
path = testQ(Q, 20, validMoves, makeMove)
print("Path for trained Q to Goal:")
for s in path:
    printState(s)

Path for trained Q to Goal:
1    
2    
3    
------
2   1
3    
------
3 2 1
------
3 1  
------
  1 3
------
1 2 3
------
1   2
    3
------


In [403]:
path

[[[1, 2, 3], [], []],
 [[2, 3], [], [1]],
 [[3], [2], [1]],
 [[3], [1, 2], []],
 [[], [1, 2], [3]],
 [[1], [2], [3]],
 [[1], [], [2, 3]]]

## Further investigation

In [436]:
Q, stepsToGoal = trainQ(5, 0.5, 0.7, validMoves, makeMove)
print('5 Repetitions')
print('State Actions in Q:' ,len(Q))
print("Progression of steps to goal:")
print(np.array(stepsToGoal))
print('Mean of steps:',mean(stepsToGoal))

5 Repetitions
State Actions in Q: 76
Progression of steps to goal:
[ 87 119  51  44  15]
Mean of steps: 63.2


In [437]:
Q, stepsToGoal = trainQ(500, 0.5, 0.7, validMoves, makeMove)
print('500 Repetitions')
print('State Actions in Q:' ,len(Q))
print("Progression of steps to goal:")
print(np.array(stepsToGoal[:100]))
print('Mean of steps:',mean(stepsToGoal))

500 Repetitions
State Actions in Q: 76
Progression of steps to goal:
[ 51  74 108  55  23  50  15  24  50  18  54  10  16   8  27   7  28  33
   9  12   7   7   9  24  11  23   7   7   7   7   8   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7]
Mean of steps: 8.144


In [438]:
Q, stepsToGoal = trainQ(10000, 0.5, 0.7, validMoves, makeMove)
print('10000 Repetitions')
print("Progression of steps to goal:")
print(np.array(stepsToGoal[:100]))
print('Mean of steps:',mean(stepsToGoal))

10000 Repetitions
Progression of steps to goal:
[118  47 118  52  16  34  16  45  36  58  10   7  26   9  15  30  12  10
  15   9  15   7   7   8   7   7  21   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7]
Mean of steps: 7.0566


In [439]:
Q, stepsToGoal = trainQ(500, .99 , 0.7, validMoves, makeMove)
print('500 Repetitions, .99 learning rate')
print("Progression of steps to goal:")
print(np.array(stepsToGoal[:100]))
print('Mean of steps:',mean(stepsToGoal))
Q, stepsToGoal = trainQ(500, .01 , 0.7, validMoves, makeMove)
print('500 Repetitions, .01 learning rate')
print("Progression of steps to goal:")
print(np.array(stepsToGoal[:100]))
print('Mean of steps:',mean(stepsToGoal))

500 Repetitions, .99 learning rate
Progression of steps to goal:
[ 24  91 125  21  37  37  22  13  16   7  11   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7]
Mean of steps: 7.654
500 Repetitions, .01 learning rate
Progression of steps to goal:
[  28  208   90  751  407  561  325  238  903  756  137 3281  188  108
   52  125   65   79  123   75  107   57  167   27   97  104   49  113
   24  118   49   90   50  116   69   63  147   22   80   55   78   46
   86  124   27  121   13   70   70   57   62   45   88   50   71  118
   16   73   84   65   42   81   35   46  118   21  122   38   35   46
   86   22   32  106   18   78   59   40   46   63   62   47   93   30
   41   69   45 

In [440]:
Q, stepsToGoal = trainQ(500, 0.5, 0.99, validMoves, makeMove)
print('500 Repetitions, .99 decay')
print("Progression of steps to goal:")
print(np.array(stepsToGoal[:100]))
print('Mean of steps:',mean(stepsToGoal))
Q, stepsToGoal = trainQ(500, 0.5, 0.01, validMoves, makeMove)
print('500 Repetitions, .01 decay')
print("Progression of steps to goal:")
print(np.array(stepsToGoal[:100]))
print('Mean of steps:',mean(stepsToGoal))

500 Repetitions, .99 decay
Progression of steps to goal:
[ 99 137  68  92 209 253 243 203  66  34  36 105 278  23  53  26  67  36
  81  26  70  54  10  58  26  21  46  96  12  31  49  18  40  19  15  55
  21  41  15  13  29  24  34  23  27  20  23  15   9  28  16  44  11  15
  11  14   8  33   9  13  11  19  39  16  19  12  17  22  11  36  18   9
  23  12  10  20  13  27  21  16  29  24  24  21  13  11  13  30  16  10
  16  10  28  10   8  14  18  22   8  12]
Mean of steps: 14.328
500 Repetitions, .01 decay
Progression of steps to goal:
[ 66  42  86 116  26  26  21  40  27  29  12  22  45  16  15  11  10  36
  19   7   7  10  10  42  10   8   7   7   7   7   7  10   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
   7   7   7   7   7   7   7   7   7   7]
Mean of steps: 8.16


In [442]:
Q, stepsToGoal = trainQ(10000, 0.99, 0.01, validMoves, makeMove)
print('10000 Repetitions, .99 learning rate, .01 decay')
print("Progression of steps to goal:")
print(np.array(stepsToGoal[:100]))
print('Mean of steps:',mean(stepsToGoal))

10000 Repetitions, .99 learning rate, .01 decay
Progression of steps to goal:
[66 42 62 36 80 47 11 15 10 24  7 14  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7]
Mean of steps: 7.033


## Grading

Download and extract `A4grader.py` from [A4grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A4grader.tar).

In [348]:
%run -i A4grader.py


======================= Code Execution =======================

Extracting python code from notebook named 'Armijo-A4.ipynb' and storing in notebookcode.py
Removing all statements that are not function or class defs or import statements.

Testing validMoves([[1], [2], [3]])

--- 10/10 points. Correctly returned [[1, 2], [1, 3], [2, 3]]

Testing validMoves([[], [], [1, 2, 3]])

--- 10/10 points. Correctly returned [[3, 1], [3, 2]]

Testing makeMove([[], [], [1, 2, 3]], [3, 2])

--- 10/10 points. Correctly returned [[], [1], [2, 3]]

Testing makeMove([[2], [3], [1]], [1, 2])

--- 10/10 points. Correctly returned [[], [2, 3], [1]]

Testing   Q, steps = trainQ(1000, 0.5, 0.7, validMoves, makeMove).

--- 10/10 points. Q dictionary has correct number of entries.

--- 10/10 points. The mean of the number of steps of 7.578 is correctly < 10.

Testing   path = testQ(Q, 20, validMoves, makeMove).

--- 20/20 points. Correctly returns path of length 7, which is correctly less than 10.

A4 Executi

## Extra Credit

Modify your code to solve the Towers of Hanoi puzzle with 4 disks instead of 3.  Name your functions

    - printState_4disk
    - validMoves_4disk
    - makeMove_4disk

Find values for number of repetitions, learning rate, and epsilon decay factor for which trainQ learns a Q function that testQ can use to find the shortest solution path.  Include the output from the successful calls to trainQ and testQ.